## Configuration and Setup


In [1]:
import os
os.environ['HF_HOME'] = "D:/HF_CACHE/"

In [2]:
from cs324_project.datasets import GlueDatasetTask, load_glue_dataset_info
from cs324_project.models import ModelCheckpointName, load_classification_model, load_pretraining_model, load_tokenizer
from cs324_project.training import get_training_args, get_trainer, get_latest_checkpoint_path

In [3]:
model_name = ModelCheckpointName.TINYBERT_HUAWEI
task = GlueDatasetTask.COLA

In [4]:
tokenizer = load_tokenizer(model_name)
dataset_info = load_glue_dataset_info(task, tokenizer)

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to D:/HF_CACHE/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

Map:   0%|          | 0/1063 [00:00<?, ? examples/s]

## Fine-tune with masking

In [5]:
model_mlm = load_pretraining_model(model_name, dataset_info)

AttributeError: 'str' object has no attribute 'value'

In [ ]:
training_args = get_training_args(task, num_epochs=1)
trainer = get_trainer(dataset_info, model_mlm, training_args)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
checkpoint_path = get_latest_checkpoint_path(training_args)

## Fine-tune on task

In [ ]:
model_sc = load_classification_model(checkpoint_path, dataset_info)

In [ ]:
training_args = get_training_args(task, num_epochs=1)
trainer = get_trainer(dataset_info, model_sc, training_args)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [6]:
from cs324_project.datasets import *
from cs324_project.models import *
from cs324_project.training import *

In [7]:
dataset = load_dataset("wikipedia", "20220301.en", split='train')

Downloading:   0%|          | 0.00/15.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/20.3G [00:00<?, ?B/s]

KeyboardInterrupt: 